In [ ]:
# Before:
@tvm.script.ir_module
class Module:
    @R.function
    def main(x: Tensor((1, 784), "float32"), w0: Tensor((784, 128), "float32"), b0: Tensor((128,), "float32"), w1: Tensor((128, 10), "float32"), b1: Tensor((10,), "float32"), label: Tensor((1, 10), "float32")) -> Tuple(Tensor(None, "float32", ndim = 2), Tensor(None, "float32", ndim = 0)):
        # block 0
        with R.dataflow():
            lv0: Tensor((1, 784), "float32") = relax.matmul(x, w0)
            lv1: Tensor(R.call_packed("vm.binary_broadcast_shape_infer", lv0, b0), "float32") = relax.add(lv0, b0)
            lv2: Tensor(None, "float32", ndim = 2) = relax.nn.relu(lv1)
            lv3: Tensor(None, "float32", ndim = 2) = relax.matmul(lv2, w1)
            out: Tensor(None, "float32", ndim = 2) = relax.add(lv3, b1)
            loss: Tensor(None, "float32", ndim = 0) = relax.nn.softmax_cross_entropy(out, label)
            R.output(out, loss)
        return (out, loss)


# After:
@tvm.script.ir_module
class Module:
    @R.function
    def main(x: Tensor((1, 784), "float32"), w0: Tensor((784, 128), "float32"), b0: Tensor((128,), "float32"), w1: Tensor((128, 10), "float32"), b1: Tensor((10,), "float32"), label: Tensor((1, 10), "float32")) -> Tuple(Tuple(Tensor(None, "float32", ndim = 2), Tensor(None, "float32", ndim = 0)), Tensor(None, "float32", ndim = 2), Tensor(None, "float32", ndim = 1), Tensor(None, "float32", ndim = 2), Tensor(None, "float32", ndim = 1)):
        # block 0
        with R.dataflow():
            lv0: Tensor((1, 784), "float32") = relax.matmul(x, w0)
            lv1: Tensor(R.call_packed("vm.binary_broadcast_shape_infer", lv0, b0), "float32") = relax.add(lv0, b0)
            lv2: Tensor(None, "float32", ndim = 2) = relax.nn.relu(lv1)
            lv3: Tensor(None, "float32", ndim = 2) = relax.matmul(lv2, w1)
            out: Tensor(None, "float32", ndim = 2) = relax.add(lv3, b1)
            loss: Tensor(None, "float32", ndim = 0) = relax.nn.softmax_cross_entropy(out, label)
            loss_adjoint: Tensor(relax.shape_of(loss), "float32") = relax.ones_like(loss)
            lv: Tensor(None, "float32", ndim = 2) = relax.nn.softmax(out)
            out_adjoint: Tensor(relax.shape_of(out), "float32") = relax.sub(lv, label)
            lv3_adjoint: Tensor(relax.shape_of(lv3), "float32") = relax.collapse_sum_like(out_adjoint, lv3)
            lv11: Tensor((10, 128), "float32") = relax.transpose(w1)
            lv21: Tensor(None, "float32", ndim = 2) = relax.matmul(lv3_adjoint, lv11)
            lv2_adjoint: Tensor(relax.shape_of(lv2), "float32") = relax.collapse_sum_like(lv21, lv2)
            lv31: Tensor(None, "float32", ndim = 2) = relax.nn.gradrelu_(lv1)
            lv1_adjoint: Tensor(R.call_packed("vm.binary_broadcast_shape_infer", lv0, b0), "float32") = relax.multiply(lv2_adjoint, lv31)
            lv0_adjoint: Tensor((1, 784), "float32") = relax.collapse_sum_like(lv1_adjoint, lv0)
            lv4: Tensor((784, 1), "float32") = relax.transpose(x)
            lv5: Tensor((784, 1), "float32") = relax.matmul(lv4, lv0_adjoint)
            w0_adjoint: Tensor((784, 128), "float32") = relax.collapse_sum_like(lv5, w0)
            b0_adjoint: Tensor((128,), "float32") = relax.collapse_sum_like(lv1_adjoint, b0)
            lv6: Tensor(None, "float32", ndim = 2) = relax.transpose(lv2)
            lv7: Tensor(None, "float32", ndim = 2) = relax.matmul(lv6, lv3_adjoint)
            w1_adjoint: Tensor((128, 10), "float32") = relax.collapse_sum_like(lv7, w1)
            b1_adjoint: Tensor((10,), "float32") = relax.collapse_sum_like(out_adjoint, b1)
            R.output(out, loss, w0_adjoint, b0_adjoint, w1_adjoint, b1_adjoint)
        return ((out, loss), w0_adjoint, b0_adjoint, w1_adjoint, b1_adjoint)